<a href="https://colab.research.google.com/github/anujsahani01/EngMarBART/blob/main/Bidirectional(mr_en).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install --upgrade accelerate
!pip install wandb
!pip install --no-cache-dir transformers sentencepiece
!pip install evaluate
!pip install rouge_score
!pip install sacremoses
!pip install fasttext
!pip install sacrebleu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!huggingface-cli login

In [ ]:
from datasets import load_dataset
dataset = load_dataset("anujsahani01/processed_en_mr")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['english', 'marathi'],
        num_rows: 2637962
    })
    test: Dataset({
        features: ['english', 'marathi'],
        num_rows: 879321
    })
})

In [ ]:
from transformers import AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50", src_lang="mr_IN", tgt_lang="en_XX", model_max_length = 512)
tokenizer.bos_token = tokenizer.eos_token

In [ ]:
tokenizer

In [ ]:
special_tokens_dict = {"eos_token": "<CLS>"}

num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print("We have added", num_added_toks, "tokens")

In [ ]:
out = tokenizer(dataset['train']['marathi'][0], text_target =  dataset['train']['english'][0],  max_length= 20, padding = 'max_length', truncation=True, return_length = True, return_tensors = 'pt')

In [ ]:
len(out['input_ids'][0])

In [ ]:
decoded_preds = tokenizer.batch_decode(out['input_ids'])
print(decoded_preds)

In [ ]:
tokenizer.batch_decode(out['labels'])

In [ ]:
import numpy as np

In [ ]:
max_length = 80
context_length = 80

def tokenize(data):
  data = tokenizer(data['marathi'], text_target =  data['english'],  max_length= max_length, padding = 'max_length', truncation=True, return_length = True, return_tensors = 'pt')
  inp_ids = []
  if len(data['input_ids']) != 0:
    for length, ids in zip(data["length"], data["input_ids"]):
      inp_ids.append(ids)
    label = []
    for length, lb in zip(data["length"], data["labels"]):
      label.append(lb)


  return {"input_ids": inp_ids,
          'labels' : label}

In [ ]:
tokenized_dataset = dataset.map(
    tokenize,
    batched=True,
    remove_columns =  dataset["train"].column_names,
)

In [ ]:
tokenized_dataset

In [ ]:
inp = tokenized_dataset['train']['input_ids']

In [ ]:
arr = []
df_final = pd.DataFrame()
df_final['tokenized_eng'] = inp

In [ ]:
df_final['length'] = df_final['tokenized_eng'].apply(
    lambda row: len(row)
)

In [ ]:
print(df_final['length'].min())
print(df_final['length'].max())
print(df_final['length'].min())

In [ ]:
! pip install -U neptune transformers

In [ ]:
from getpass import getpass
from transformers.integrations import NeptuneCallback
import neptune
token=my_api_token = getpass("Enter your Neptune API token:")

In [ ]:
my_project = "anujsahani01/ITStudio"
run = neptune.init_run(
    project="anujsahani01/ITStudio",
    api_token = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJkMzNmZWY5My0yOGY1LTRiZDItODA1MS05ZTM3MTc0MTI2NTYifQ==',
    source_files= ["/content/drive/MyDrive/Mbart(Finetuning).ipynb"]
)


neptune_callback = NeptuneCallback(run=run)
# run["train_dataset"].track_files("./train.csv")


In [ ]:
from transformers import AutoModel

from transformers import AutoModelForSeq2SeqLM, AutoConfig
model_checkpoint  = "anujsahani01/finetuned_en2mr"
config = AutoConfig.from_pretrained(model_checkpoint, vocab_size=len(tokenizer), use_auth_token=True,)

config.vocab_size = tokenizer.vocab_size
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, config = config, ignore_mismatched_sizes=True).to('cuda')
model.resize_token_embeddings(len(tokenizer))

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
! pip install evaluate

In [ ]:
import evaluate
metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
  preds, labels = eval_preds
  # In case the model returns more than the prediction logits
  if isinstance(preds, tuple):
      preds = preds[0]

  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

  # Replace -100s in the labels as we can't decode them
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # Some simple post-processing
  decoded_preds = [pred.strip() for pred in decoded_preds]
  decoded_labels = [[label.strip()] for label in decoded_labels]

  result = metric.compute(predictions=decoded_preds, references=decoded_labels)
  return {"bleu": result["score"]}

In [ ]:
params = {'lr':0.001, 'max_steps': 10000, 'warmup_steps': 50, 'weight_decay': 0.01, 'per_device_train_batch_size': 12, 'per_device_eval_batch_size': 12, 'evaluation_strategy' :"no", 'save_total_limit' :1, 'num_train_epochs': 1, 'predict_with_generate' : True, 'fp16' : False, 'push_to_hub' : True, 'remove_unused_columns' :False, 'dropout': 0.5}
# params = {'lr':0.001, 'max_steps': 8000, 'warmup_steps': 50, 'weight_decay': 0.01, 'per_device_train_batch_size': 12, 'per_device_eval_batch_size': 12, 'evaluation_strategy' :"no", 'save_total_limit' :1, 'num_train_epochs': 1, 'predict_with_generate' : True, 'fp16' : False, 'push_to_hub' : True, 'remove_unused_columns' :False, 'dropout': 0.5}

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

args = Seq2SeqTrainingArguments(
    f"./bidirectional_translator",
    evaluation_strategy= params['evaluation_strategy'],
    save_strategy="epoch",
    learning_rate= params['lr'] ,
    per_device_train_batch_size=params['per_device_train_batch_size'],
    per_device_eval_batch_size= params['per_device_eval_batch_size'],
    warmup_steps = params['warmup_steps'],
    weight_decay= params['weight_decay'],
    save_total_limit= params['save_total_limit'],
    num_train_epochs= params['num_train_epochs'],
    predict_with_generate= params['predict_with_generate'],
    fp16= params['fp16'],
    push_to_hub= params['push_to_hub'],
    max_steps = params['max_steps'],
    remove_unused_columns= params['remove_unused_columns'],
    report_to="neptune",
    # predict_with_generate = True
)

In [ ]:
neptune_callback = NeptuneCallback(run=run)

In [ ]:
trainer = Seq2SeqTrainer(
    model.to('cuda'),
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,)
    # callbacks=[neptune_callback])

In [ ]:
for cb in trainer.callback_handler.callbacks:
  if isinstance(cb, NeptuneCallback):
    trainer.callback_handler.remove_callback(cb)

In [ ]:
trainer.train()

In [ ]:
metrics = trainer.state.log_history

In [ ]:
for key, value in params.items():
  run[key] = value

In [ ]:
lr = []
train_loss = []
steps =  []
epoch = []

In [ ]:
for i in range(len(metrics)-1):
  train_loss.append(metrics[i]['loss'])
  steps.append(metrics[i]['step'])
  lr.append(metrics[i]['learning_rate'])
  epoch.append(metrics[i]['epoch'])

In [ ]:
for i in range(len(train_loss)):
  run['train/loss'].append(train_loss[i])

for i in range(len(epoch)):
  run['epoch'].append(epoch[i])

for i in range(len(lr)):
  run["learning_rate"].append(lr[i])

for i in range(len(steps)):
  run["steps"].append(steps[i])

In [ ]:
trainer.push_to_hub()

In [ ]:
! pip install sacremoses